In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
Updating files:   7% (1042/13996)
Updating files:   8% (1120/13996)
Updating files:   9% (1260/13996)
Updating files:  10% (1400/13996)
Updating files:  11% (1540/13996)
Updating files:  12% (1680/13996)
Updating files:  13% (1820/13996)
Updating files:  14% (1960/13996)
Updating files:  14% (2088/13996)
Updating files:  15% (2100/13996)
Updating files:  16% (2240/13996)
Updating files:  17% (2380/13996)
Updating files:  18% (2520/13996)
Updating files:  19% (2660/13996)
Updating files:  20% (2800/13996)
Updating files:  21% (2940/13996)
Updating files:  22% (3080/13996)
Updating files:  23% (3220/13996)
Updating files:  24% (3360/13996)
Updating files:  24% (3423/13996)
Updating files:  25% (3499/13996)
Updating files:  26% (3639/13996)
Updating files:  27% (3779/13996)
Updating files:  28% (3919/13996)
Updating files:  29% (4059/13996)
Updating files:  30% (4199/13996)
Updating files:  31% (4339/13996)
Updating files:  32% (4479/13996)
Updating fi

In [9]:
import csv
data = {}
with open('facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [10]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [11]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'master_data'

In [12]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'master_data/training/anger'

In [13]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('facial_expressions/images', image)
    dest = os.path.join('master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('facial_expressions/images', image)
    dest = os.path.join('master_data/testing', emotion, image)
    copyfile(source, dest)

In [15]:
pip install tensorflow

     ---------------------------------------- 0.0/272.9 MB ? eta -:--:--
     --------------------------------------- 0.4/272.9 MB 12.2 MB/s eta 0:00:23
     --------------------------------------- 1.2/272.9 MB 15.8 MB/s eta 0:00:18
     --------------------------------------- 1.8/272.9 MB 14.5 MB/s eta 0:00:19
     --------------------------------------- 2.2/272.9 MB 12.6 MB/s eta 0:00:22
     --------------------------------------- 2.6/272.9 MB 11.8 MB/s eta 0:00:23
     --------------------------------------- 2.7/272.9 MB 10.2 MB/s eta 0:00:27
     ---------------------------------------- 2.9/272.9 MB 9.1 MB/s eta 0:00:30
      --------------------------------------- 3.6/272.9 MB 9.4 MB/s eta 0:00:29
      --------------------------------------- 4.0/272.9 MB 9.9 MB/s eta 0:00:28
      -------------------------------------- 4.7/272.9 MB 10.4 MB/s eta 0:00:26
      -------------------------------------- 5.3/272.9 MB 10.5 MB/s eta 0:00:26
      -------------------------------------- 5.

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [20]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

In [17]:
train_dir = 'master_data/training'
test_dir = 'master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [18]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [21]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

C:\Users\91907\AppData\Local\Temp\ipykernel_4028\1393094347.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 88s 1s/step - loss: 0.9505 - acc: 0.6015 - val_loss: 1.4546 - val_acc: 0.5594
Epoch 2/10
86/86 [==============================] - 45s 528ms/step - loss: 0.5883 - acc: 0.7976 - val_loss: 1.7386 - val_acc: 0.6579
Epoch 3/10
86/86 [==============================] - 47s 547ms/step - loss: 0.5117 - acc: 0.8238 - val_loss: 1.6696 - val_acc: 0.6648
Epoch 4/10
86/86 [==============================] - 45s 525ms/step - loss: 0.4609 - acc: 0.8367 - val_loss: 1.5587 - val_acc: 0.6699
Epoch 5/10
86/86 [==============================] - 46s 532ms/step - loss: 0.4306 - acc: 0.8495 - val_loss: 1.9288 - val_acc: 0.6838
Epoch 6/10
86/86 [==============================] - 45s 524ms/step - loss: 0.3957 - acc: 0.8618 - val_loss: 2.3867 - val_acc: 0.6678
Epoch 7/10
86/86 [==============================] - 45s 526ms/step - loss: 0.3610 - acc: 0.8713 - val_loss: 1.4302 - val_acc: 0.6820
